<a href="https://colab.research.google.com/github/XavierCarrera/text-classification-system/blob/main/Hidden_Markovian_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Exploration

In [ ]:
!pip install conllu
!git clone https://github.com/UniversalDependencies/UD_Spanish-AnCora.git

In [ ]:
from conllu import parse_incr 
wordList = []
data_file = open("UD_Spanish-AnCora/es_ancora-ud-dev.conllu", "r", encoding="utf-8")
for tokenlist in parse_incr(data_file):
    print(tokenlist.serialize())

Se truncaron las últimas líneas 5000 del resultado de transmisión.
17	un	uno	DET	DET	Definite=Ind|Gender=Masc|Number=Sing|PronType=Art	18	det	_	_
18	mes	mes	NOUN	NOUN	Gender=Masc|Number=Sing	16	nsubj	_	_
19	de	de	ADP	ADP	AdpType=Prep	20	case	_	_
20	Febrero	Febrero	NOUN	NOUN	AdvType=Tim	18	nmod	_	_
21	muy	mucho	ADV	ADV	_	22	advmod	_	_
22	apretado	apretado	ADJ	ADJ	Gender=Masc|Number=Sing|VerbForm=Part	18	amod	_	SpaceAfter=No
23	"	"	PUNCT	PUNCT	PunctType=Quot	4	punct	_	SpaceAfter=No
24	,	,	PUNCT	PUNCT	PunctType=Comm	4	punct	_	_
25	explicó	explicar	VERB	VERB	Mood=Ind|Number=Sing|Person=3|Tense=Past|VerbForm=Fin	0	root	_	SpaceAfter=No
26	.	.	PUNCT	PUNCT	PunctType=Peri	25	punct	_	_


# sent_id = dev-s916
# text = Al margen de que, como todos, han disfrutado de unos días de descanso, curiosamente Rivaldo y Overmars tampoco estuvieron en Gandía con lo que esa elección resulta más que cuestionable.
1	Al	al	ADP	ADP	AdpType=Preppron	10	mark	_	MWE=Al_margen_de|MWEPOS=ADP
2	margen	margen	NOUN	NOUN	

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
tokenlist[1]

{'deprel': 'nsubj',
 'deps': None,
 'feats': {'Gender': 'Masc', 'Number': 'Sing'},
 'form': 'cierto',
 'head': 3,
 'id': 2,
 'lemma': 'cierto',
 'misc': None,
 'upos': 'ADJ',
 'xpos': 'ADJ'}

In [ ]:
tokenlist[1]['form']+'|'+tokenlist[1]['upos']

'cierto|ADJ'

# Model Training

In [ ]:
tagCountDict = {} 
emissionDict = {}
transitionDict = {}

tagtype = 'upos'
data_file = open("UD_Spanish-AnCora/es_ancora-ud-dev.conllu", "r", encoding="utf-8")

for tokenlist in parse_incr(data_file):
  prevtag = None
  for token in tokenlist:

    # C(tag)
    tag = token[tagtype]
    if tag in tagCountDict.keys():
      tagCountDict[tag] += 1
    else:
      tagCountDict[tag] = 1

    # C(word|tag)
    wordtag = token['form'].lower()+'|'+token[tagtype] # (word|tag)
    if wordtag in emissionDict.keys():
      emissionDict[wordtag] = emissionDict[wordtag] + 1
    else:
      emissionDict[wordtag] = 1

    #  C(tag|previous_tag) 
    if prevtag is None:
      prevtag = tag
      continue
    transitiontags = tag+'|'+prevtag
    if transitiontags in transitionDict.keys():
      transitionDict[transitiontags] = transitionDict[transitiontags] + 1
    else:
      transitionDict[transitiontags] = 1
    prevtag = tag

In [ ]:
transitionProbDict = {} # matrix A
emissionProbDict = {} # matrix B

# transition Probabilities 
for key in transitionDict.keys():
  tag, prevtag = key.split('|')
  if tagCountDict[prevtag]>0:
    transitionProbDict[key] = transitionDict[key]/(tagCountDict[prevtag])
  else:
    print(key)

# emission Probabilities 
for key in emissionDict.keys():
  word, tag = key.split('|')
  if emissionDict[key]>0:
    emissionProbDict[key] = emissionDict[key]/tagCountDict[tag]
  else:
    print(key)

transitionProbDict['ADJ|ADJ']

0.030225988700564973

In [ ]:
import numpy as np
np.save('transitionHMM.npy', transitionProbDict)
np.save('emissionHMM.npy', emissionProbDict)
transitionProbdict = np.load('transitionHMM.npy', allow_pickle='TRUE').item()
transitionProbDict['ADJ|ADJ']

0.030225988700564973